### Data collection
In this notebook, we pull stock and company financials data from alpha vantage.
First, we import the necessary packages below.

In [2]:
#import usual packages for data manipulation
import pandas as pd
import numpy as np

#import usual packages for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#import datetime to get the current date
import datetime as dt

#import alpha vantage packages
from alpha_vantage.timeseries import TimeSeries #for stock data
from alpha_vantage.techindicators import TechIndicators #for technical indicators
from alpha_vantage.fundamentaldata import FundamentalData #for company overview
from alpha_vantage.alphaintelligence import AlphaIntelligence #for news sentiment

To use alpha vantage, we need an API key, which can be obtained at this link:
<https://www.alphavantage.co/support/#api-key>

Note: the free API key allows up to 25 queries per day.

After obtaining our personal key, we initialize the four classes from which we will pull data.

In [6]:
#Arvind's API key
my_key='DJ3QKKTFZ5J298QY'

ts = TimeSeries(key=my_key, output_format='pandas')
fd = FundamentalData(key=my_key, output_format='pandas')
ai = AlphaIntelligence(key=my_key, output_format='pandas')

Next, we write a few simple functions that will pull the data, apply some pre-processing, and then save to csv.

In [7]:
#function to get stock data and save to csv
def stock_to_csv(ticker):
    df, _ = ts.get_daily(symbol=ticker, outputsize='full')
    df.columns = ['open','high','low','close','volume']
    df.sort_index(inplace=True)
    df = df[df.index.year >= 2014]
    df.to_csv(f'../data/{ticker}_stock.csv')

In [8]:
def ticker_news_to_csv(ticker):
    df, _ = ai.get_news_sentiment(tickers = ticker, time_from='20220810T0130', limit=1000)
    df.reset_index(inplace=True, drop=True)
    df.to_csv(f'../data/{ticker}_news_data_raw.csv')
    #choose only the date and news sentiment score columns
    df = pd.DataFrame({'date': pd.to_datetime(df['time_published']).dt.date,
                       'ticker_sentiment': df['ticker_sentiment']})
    df = df.groupby('date').sum()
    scores = [] #to store daily sentiment scores for aapl
    relevances = [] #to store daily relevance scores for aapl
    for i in range(len(df)):
        temp = pd.DataFrame(df['ticker_sentiment'].iloc[i])
        temp = temp[temp['ticker'] == ticker]
        wts = temp['relevance_score'].astype(float)
        raw_scores = temp['ticker_sentiment_score'].astype(float)
        #append the mean relevance score for the day
        relevances.append(wts.mean())
        #append the weighted average sentiment score for the day
        scores.append(np.dot(wts,raw_scores) / wts.sum())
    df[f'{ticker}_sentiment_score'] = np.array(scores)
    df[f'{ticker}_relevance_score'] = np.array(relevances)
    df.drop('ticker_sentiment', axis=1, inplace=True)
    df.to_csv(f'../data/{ticker}_news_data.csv')

In [9]:
def financial_cleaner(df):
    df['date'] = pd.to_datetime(df['fiscalDateEnding'])
    df.drop(columns='fiscalDateEnding',inplace=True)
    df.set_index('date',inplace=True)
    df.sort_index(inplace=True)
    df = df[df.index.year >= 2014]
    return df

def financials_to_csv(ticker):
    df_e, _ = fd.get_earnings_quarterly(ticker)
    df_b, _ = fd.get_balance_sheet_quarterly(ticker)
    df_i, _ = fd.get_income_statement_quarterly(ticker)
    df_e.drop(columns=['reportTime'],inplace=True)
    df_b.drop(columns=['reportedCurrency'], inplace=True)
    df_i.drop(columns=['reportedCurrency'], inplace=True)
    #cleaning the data
    df_b = financial_cleaner(df_b)
    df_i = financial_cleaner(df_i)
    df_e = financial_cleaner(df_e)
    #concatenating the dataframes
    df_financials = pd.concat([df_b,df_i,df_e],axis=1)
    df_financials.to_csv(f'../data/{ticker}_financials.csv')

Let's check that it all works by setting the ticker to NVDA.

In [10]:
ticker = 'NVDA'
stock_to_csv(ticker)
ticker_news_to_csv(ticker)
financials_to_csv(ticker)

To read in the csv's, some care is needed to ensure that the dtypes of the columns are correctly interpreted. For convenience, we wrap this in a simple function that pulls the data for a given ticker.

In [13]:
def get_data(ticker):
    df_stock = pd.read_csv(f'../data/{ticker}_stock.csv', index_col='date',parse_dates=True,dtype=float)
    df_news = pd.read_csv(f'../data/{ticker}_news_data.csv', index_col='date',parse_dates=True,dtype=float)
    df_financials = pd.read_csv(f'../data/{ticker}_financials.csv', 
                                index_col='date',
                                parse_dates=True,
                                dtype={col: 'float' for col in pd.read_csv(f'../data/{ticker}_financials.csv', nrows=1).columns if col!='reportedDate'},
                                na_values='None')
    return {'stock': df_stock, 
            'news' : df_news, 
            'financials' : df_financials}

In [14]:
nvda = get_data('NVDA')

In [18]:
nvda['financials']

,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,...,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
date,,,,,,,,,,,,,,,,,,,,,
2014-01-26,7.250894e+09,5.624711e+09,1.152000e+09,4.671810e+09,3.877650e+08,NaN,1.626183e+09,5.827400e+08,7.000800e+08,9.391910e+08,...,6.192420e+08,1.460460e+08,1.761160e+08,2.308060e+08,1.469170e+08,NaN,NaN,NaN,NaN,NaN
2014-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2014-02-12,0.006,0.005,0.001,20.0000
2014-04-27,6.864663e+09,5.273798e+09,5.090000e+08,4.347817e+09,3.932800e+08,NaN,1.590865e+09,5.708020e+08,NaN,9.207090e+08,...,1.365160e+08,1.380000e+08,1.740000e+08,2.290000e+08,1.370000e+08,NaN,NaN,NaN,NaN,NaN
2014-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2014-05-06,0.006,0.004,0.002,50.0000
2014-07-27,6.933719e+09,5.377586e+09,5.150000e+08,4.386054e+09,3.874340e+08,NaN,1.556133e+09,5.569110e+08,NaN,9.037920e+08,...,1.279760e+08,1.260000e+08,1.670000e+08,2.230000e+08,1.280000e+08,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023-11-21,0.402,0.336,0.066,19.6429
2024-01-28,6.572800e+10,4.434500e+10,7.280000e+09,2.598400e+10,5.282000e+09,9.999000e+09,2.289900e+10,3.914000e+09,3.509000e+09,5.512000e+09,...,1.228500e+10,1.240000e+10,1.416900e+10,1.431200e+10,1.228500e+10,NaN,NaN,NaN,NaN,NaN
2024-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2024-02-21,0.516,0.461,0.055,11.9306


In [19]:
ticker = 'TSLA'
stock_to_csv(ticker)
ticker_news_to_csv(ticker)
financials_to_csv(ticker)

In [20]:
tsla = get_data('TSLA')

In [32]:
tsla['financials']

,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,...,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
date,,,,,,,,,,,,,,,,,,,,,
2014-03-31,4.500410e+09,3.156047e+09,2.393908e+09,2.584068e+09,4.507300e+08,7.238000e+07,8.926340e+08,8.493890e+08,1.693350e+08,13662000.0,...,-4.980000e+07,-4.980000e+07,-3.710800e+07,7.160000e+06,-4.980000e+07,2014-05-07,0.01,0.10,-0.09,-90.0000
2014-06-30,5.054463e+09,3.441694e+09,2.674910e+09,2.686624e+09,5.969270e+08,9.660700e+07,1.081543e+09,1.035822e+09,2.046900e+08,13541000.0,...,-6.190000e+07,-6.190000e+07,-2.951200e+07,2.520300e+07,-6.190000e+07,2014-07-31,0.01,0.04,-0.03,-75.0000
2014-09-30,5.437533e+09,3.362914e+09,2.370735e+09,2.370735e+09,7.524920e+08,1.568890e+08,1.470296e+09,1.404326e+09,2.434470e+08,13420000.0,...,-7.470800e+07,-7.470800e+07,-4.191900e+07,2.305300e+07,-7.470800e+07,2014-11-05,0.02,-0.01,0.03,300.0000
2014-12-31,5.830667e+09,3.180073e+09,1.905713e+09,1.905713e+09,9.536750e+08,2.266040e+08,1.883850e+09,1.829267e+09,2.925900e+08,NaN,...,2.616970e+08,-1.076510e+08,-7.520800e+07,-7.232000e+06,-1.076300e+08,2015-02-11,-0.01,0.02,-0.03,-150.0000
2015-03-31,6.120030e+09,2.921417e+09,1.510076e+09,1.510076e+09,1.054840e+09,2.000520e+08,2.286552e+09,2.224191e+09,3.423860e+08,NaN,...,-1.541810e+08,-1.701240e+08,-1.245670e+08,-4.745500e+07,-1.541810e+08,2015-05-06,-0.36,-0.50,0.14,28.0000
2015-06-30,6.468185e+09,2.628621e+09,1.150673e+09,1.150673e+09,1.212279e+09,1.386480e+08,2.719318e+09,2.646017e+09,4.038690e+08,NaN,...,-1.842270e+08,-1.830660e+08,-1.567080e+08,-6.531900e+07,-1.842270e+08,2015-08-05,-0.03,-0.60,0.57,95.0000
2015-09-30,7.547497e+09,2.998795e+09,1.426036e+09,1.426036e+09,1.293717e+09,1.199640e+08,3.187977e+09,3.103811e+09,4.727690e+08,NaN,...,-2.298580e+08,-2.390390e+08,-1.987660e+08,-8.840000e+07,-2.298580e+08,2015-11-03,-0.04,-0.04,0.00,0.0000
2015-12-31,8.067939e+09,2.782006e+09,1.196908e+09,1.196908e+09,1.277838e+09,1.689650e+08,3.416156e+09,3.403334e+09,5.711260e+08,12816000.0,...,2.185640e+08,-2.999680e+08,-2.767320e+08,-1.330090e+08,-3.203970e+08,2016-02-10,-0.87,0.10,-0.97,-970.0000
2016-03-31,9.191702e+09,3.239543e+09,1.441789e+09,1.441789e+09,1.301961e+09,3.180560e+08,3.707949e+09,3.593014e+09,6.730890e+08,NaN,...,-2.822670e+08,-2.651460e+08,-2.377960e+08,-8.133600e+07,-2.822670e+08,2016-05-04,-0.04,-0.05,0.01,20.0000


In [26]:
ticker = 'META'
stock_to_csv(ticker)
ticker_news_to_csv(ticker)
financials_to_csv(ticker)

In [27]:
meta = get_data('META')

In [30]:
meta['news']

,META_sentiment_score,META_relevance_score
date,,
2024-08-04,0.034152,0.087244
2024-08-05,0.106548,0.129432
2024-08-06,0.216477,0.124139
2024-08-07,0.190669,0.109720
2024-08-08,0.059341,0.076973
2024-08-09,0.124186,0.094386
2024-08-10,0.019719,0.095671
2024-08-11,0.118520,0.094934
2024-08-12,0.185444,0.113785


In [31]:
ticker = 'NFLX'
stock_to_csv(ticker)
ticker_news_to_csv(ticker)
financials_to_csv(ticker)

In [33]:
nflx = get_data('NFLX')

In [36]:
nflx['news']

,NFLX_sentiment_score,NFLX_relevance_score
date,,
2024-06-26,0.176093,0.036460
2024-06-27,0.230301,0.187146
2024-06-28,0.134642,0.164873
2024-06-29,0.092957,0.140604
2024-06-30,0.114813,0.155233
2024-07-01,-0.043832,0.203937
2024-07-02,0.143280,0.209403
2024-07-03,0.116522,0.239829
2024-07-04,0.123569,0.241946
